# Grant's attempt at making sense of SegmentFlow outputs and grain statistics (a two-for-one special)

In [26]:
import numpy as np
import matplotlib.pyplot as plt
from stl import mesh
%load_ext autoreload
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
file = '../../segmentflow_02/stls/isolated-classes_00330.stl'
m = mesh.Mesh.from_file(file)
vertices = m.vectors.reshape((-1, 3))
center = vertices.mean(axis=0)
centered = vertices - center


# # Testing:
# # generate points on an ellipsoid
# N = 10
# theta = np.linspace(0, 2*np.pi, N)
# phi = np.linspace(0, np.pi, N)
# x = 5.0*np.outer(np.cos(theta), np.sin(phi))
# y = 2*np.outer(np.sin(theta), np.sin(phi))
# z = np.outer(np.ones(np.size(theta)), np.cos(phi))
# # flatten the arrays
# x = x.flatten()
# y = y.flatten()
# z = z.flatten()
# # stack the arrays
# centered = np.vstack((x, y, z)).T

# Find the principal ellipse
cov = np.cov(centered.T)
evals,evecs = np.linalg.eig(cov)

In [114]:
# Generate points along the principal ellipsoid
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 50)
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones_like(u), np.cos(v))
points = np.stack([x.reshape(-1), y.reshape(-1), z.reshape(-1)], axis=1)

# Scale and rotate the points by the eigenvectors and eigenvalues
scaled_points = np.dot(points * np.sqrt(evals) * 2, evecs)
x = scaled_points[:, 0]
y = scaled_points[:, 1]
z = scaled_points[:, 2] 

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')
ax.scatter(x, y, z, color='r', alpha=0.1)
ax.scatter(centered[:, 0], centered[:, 1], centered[:, 2], color='b', alpha=0.1)
# set equal axes scales
max_range = np.array([x.max()-x.min(), y.max()-y.min(), z.max()-z.min()]).max() / 2.0
mid_x = (x.max()+x.min()) * 0.5
mid_y = (y.max()+y.min()) * 0.5
mid_z = (z.max()+z.min()) * 0.5
max_range *= 1.2
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

# ToDo: plot surface and wireframe

# plot surface and wireframe
# ax.plot_surface(x, y, z,  rstride=4, cstride=4, color='b', alpha=0.1)
# ax.plot_wireframe(x, y, z,  rstride=4, cstride=4, color='k', alpha=0.3)
# add shadows
# ax.contour(x, y, z, zdir='x', offset=-1, cmap='Blues')
# ax.contour(x, y, z, zdir='y', offset=1, cmap='Blues')
# ax.contour(x, y, z, zdir='z', offset=-1, cmap='Blues')



(-0.023690900712230623, 0.02369315760201595)